<a href="https://colab.research.google.com/github/nataliakro02/25.03/blob/main/NK_System_rekomendacyjny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

In [1]:
!pip install numpy==1.23.5

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2461573 sha256=880d1d40bb8680d75cffed8a70a8d9f360464fd2717f1a9275b16f432ee78c78
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
import numpy as np
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.prediction_algorithms.knns import KNNBasic, KNNBaseline

In [17]:
movie = pd.read_csv('movies.csv')
rate = pd.read_csv('ratings.csv')

In [11]:
min_rating = rate.rating.min()
max_rating = rate.rating.max()

reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(rate[['userId', 'movieId', 'rating']], reader)

In [15]:
df = SVD()
cross_validate(df, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8704  0.8795  0.8763  0.8746  0.8693  0.8740  0.0038  
MAE (testset)     0.6708  0.6754  0.6725  0.6708  0.6685  0.6716  0.0023  
Fit time          1.64    1.53    1.77    1.50    1.49    1.59    0.10    
Test time         0.25    0.12    0.27    0.11    0.11    0.17    0.07    


{'test_rmse': array([0.87035773, 0.87953268, 0.87630486, 0.87462607, 0.86933913]),
 'test_mae': array([0.67076995, 0.67542148, 0.67253202, 0.67082236, 0.66845802]),
 'fit_time': (1.640251636505127,
  1.5343780517578125,
  1.7668952941894531,
  1.5014023780822754,
  1.494462013244629),
 'test_time': (0.24901700019836426,
  0.11787223815917969,
  0.27387356758117676,
  0.1102900505065918,
  0.11186552047729492)}

In [16]:
trainset = data.build_full_trainset()
df.fit(trainset)

In [28]:
wszystkie_filmy = set(movie['movieId'])
ocenione_filmy = set(rate[rate['userId'] == 600]['movieId'])
nieocenione_filmy = wszystkie_filmy - ocenione_filmy

predictions = [ (movie_id, df.predict(600, movie_id).est) for movie_id in nieocenione_filmy]
predictions.sort(key=lambda x: x[1], reverse=True)
top_10 = predictions[:10]

polecane_filmy = pd.DataFrame(top_10, columns=['movieId', 'predicted_rating'])
polecane_filmy = polecane_filmy.merge(movie, on='movieId')
print(polecane_filmy[['movieId', 'title', 'predicted_rating']])

   movieId                                              title  \
0    78499                                 Toy Story 3 (2010)   
1     1279                              Night on Earth (1991)   
2     1262                           Great Escape, The (1963)   
3    66934              Dr. Horrible's Sing-Along Blog (2008)   
4     1208                              Apocalypse Now (1979)   
5      541                                Blade Runner (1982)   
6      904                                 Rear Window (1954)   
7      293  Léon: The Professional (a.k.a. The Professiona...   
8     1221                     Godfather: Part II, The (1974)   
9      908                          North by Northwest (1959)   

   predicted_rating  
0          4.428065  
1          4.229044  
2          4.227936  
3          4.197031  
4          4.192804  
5          4.176596  
6          4.155888  
7          4.090675  
8          4.088191  
9          4.075100  
